### Ingestion of CSV & Excel (Structured Data)

In [2]:
import pandas as pd
import os

In [3]:
os.makedirs("data/structured_data", exist_ok=True)

In [4]:
#create sample data

data = {
    "Name": ["Arun", "Priya", "Kumar", "Sneha"],
    "Age": [21, 23, 22, 24],
    "Course": ["Computer Science", "Data Science", "AI & ML", "Cybersecurity"],
    "Marks": [85, 90, 88, 92],
    "Grade": ["B", "A", "A", "A+"],
    "City": ["Chennai", "Bangalore", "Hyderabad", "Mumbai"],
    "Enrollment_Year": [2022, 2021, 2022, 2020],
    "Scholarship": [True, False, True, True]
}

#save as csv

df = pd.DataFrame(data)
df.to_csv("data/structured_data/students.csv", index=False)

In [6]:
#save as excel with multiple sheets

with pd.ExcelWriter('data/structured_data/inventory.xlsx') as writer:
    df.to_excel(writer, sheet_name="products", index=False)

    #add another sheet
    summary_data = {
        "category": ["electronics", "accessories"],
        "total_items": [3,2],
        "total_value" : [1378, 2567]
    }

    pd.DataFrame(summary_data).to_excel(writer, sheet_name='summary', index=False)

In [7]:
#CSV processing - each row becomes a document

from langchain_community.document_loaders import CSVLoader, UnstructuredCSVLoader

print("CSVLoader - Row-Based Documents")
csv_loader = CSVLoader(file_path="data/structured_data/students.csv",
                       encoding='utf-8',
                       csv_args={"delimiter":",",
                                 "quotechar": '"'})

csv_docs = csv_loader.load()
print(f"Loaded {len(csv_docs)} documents (one per row)")
print("\n First Document:")
print(f"Content : {csv_docs[0]. page_content}")
print(f"Metadata : {csv_docs[0].metadata} ")


d:\RAGs In Depth\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CSVLoader - Row-Based Documents
Loaded 4 documents (one per row)

 First Document:
Content : Name: Arun
Age: 21
Course: Computer Science
Marks: 85
Grade: B
City: Chennai
Enrollment_Year: 2022
Scholarship: True
Metadata : {'source': 'data/structured_data/students.csv', 'row': 0} 


In [12]:
#custom csv processing for better control
from typing import List
from langchain_core.documents import Document

print("\n Custom CSV Processing")

def process_csv(filepath:str)->List[Document]:
    "process csv with document creation"
    df = pd.read_csv(filepath)
    documents = []

    #one document per row with structure content
    for idx, row in df.iterrows():
        #create structured content
        content = f""" Student Information:
        Name: {row['Name']}
        Age: {row['Age']}
        Course: {row['Course']}
        Marks: {row['Marks']}
        Grade: {row['Grade']}
        City: {row['City']}
        Enrollment_Year: {row['Enrollment_Year']}
        Scholarship: {row['Scholarship']}
"""
        
        #create document with rich metadata
        doc = Document(
            page_content=content,
            metadata={
                "source": filepath,
                "row_index": idx,
                "student_name" : row['Name'],
                "course": row['Course']

            }
        )

        documents.append(doc)

    return documents    


 Custom CSV Processing


In [14]:
csv_documents = process_csv("data/structured_data/students.csv")
print(f"Total Documents : {len(csv_documents)}")
print(f"First Document:\n")
print(f"Content: {csv_documents[0].page_content}")
print(f"Metadata: {csv_documents[0].metadata}")

Total Documents : 4
First Document:

Content:  Student Information:
        Name: Arun
        Age: 21
        Course: Computer Science
        Marks: 85
        Grade: B
        City: Chennai
        Enrollment_Year: 2022
        Scholarship: True

Metadata: {'source': 'data/structured_data/students.csv', 'row_index': 0, 'student_name': 'Arun', 'course': 'Computer Science'}


In [ ]:
#Excel Processing - each sheet is a document

from langchain_community.document_loaders import UnstructuredExcelLoader

try:
    excel_loader = UnstructuredExcelLoader(file_path="data/structured_data/inventory.xlsx", mode="elements")
    excel_docs = excel_loader.load()
    print(f"Number of documents: {len(excel_docs)}")
    print(excel_docs)
except Exception as e:
    print(f"Error Occurred: {e}")    

Number of documents: 2
[Document(metadata={'source': 'data/structured_data/inventory.xlsx', 'file_directory': 'data/structured_data', 'filename': 'inventory.xlsx', 'last_modified': '2025-12-07T14:48:16', 'page_name': 'products', 'page_number': 1, 'text_as_html': '<table><tr><td>Name</td><td>Age</td><td>Course</td><td>Marks</td><td>Grade</td><td>City</td><td>Enrollment_Year</td><td>Scholarship</td></tr><tr><td>Arun</td><td>21</td><td>Computer Science</td><td>85</td><td>B</td><td>Chennai</td><td>2022</td><td>True</td></tr><tr><td>Priya</td><td>23</td><td>Data Science</td><td>90</td><td>A</td><td>Bangalore</td><td>2021</td><td>False</td></tr><tr><td>Kumar</td><td>22</td><td>AI &amp; ML</td><td>88</td><td>A</td><td>Hyderabad</td><td>2022</td><td>True</td></tr><tr><td>Sneha</td><td>24</td><td>Cybersecurity</td><td>92</td><td>A+</td><td>Mumbai</td><td>2020</td><td>True</td></tr></table>', 'languages': ['eng'], 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', '